In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install openpyxl==3.0.0

     |████████████████████████████████| 172 kB 4.3 MB/s 
  Created wheel for openpyxl: filename=openpyxl-3.0.0-py2.py3-none-any.whl size=241207 sha256=7a41c9f0a2cbe672573404903ff552543f9b9ecdad1d68f9ae8721824c3ebaf6
  Stored in directory: /root/.cache/pip/wheels/c7/64/ff/ce98f6e1d2701ae8e216c875da62feed2839ac8a3cae0ab8af
Successfully built openpyxl
  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 3.0.9
    Uninstalling openpyxl-3.0.9:
      Successfully uninstalled openpyxl-3.0.9


In [ ]:
df = pd.read_excel("/content/g28.xlsx")
df

,Полное наименование ru
0,Розетка телефонная внешняя двойная RJ-12
1,Кабель Cisco/CAB-GS-1M= StackWise 1M Stacking ...
2,ТЕЛЕФОННЫЙ АППАРАТ SIEMENS 5020
3,Телефон Cisco IP Phone 7911G (CP-7911G)
4,Cистема производственно технологической громко...
...,...
9821,АДАПТЕР REDA 1
9822,Антенна
9823,ТЕЛЕФОН СОТОВЫЙ SIEMENS M 50
9824,ТЕЛЕФОН СОТОВЫЙ PHILIPS 625


In [ ]:
data = df[["Полное наименование ru"]]
data = data.rename(columns={'Полное наименование ru':"text"})
data = data.dropna(subset=['text'])

In [ ]:
data

,text
0,Розетка телефонная внешняя двойная RJ-12
1,Кабель Cisco/CAB-GS-1M= StackWise 1M Stacking ...
2,ТЕЛЕФОННЫЙ АППАРАТ SIEMENS 5020
3,Телефон Cisco IP Phone 7911G (CP-7911G)
4,Cистема производственно технологической громко...
...,...
9821,АДАПТЕР REDA 1
9822,Антенна
9823,ТЕЛЕФОН СОТОВЫЙ SIEMENS M 50
9824,ТЕЛЕФОН СОТОВЫЙ PHILIPS 625


In [ ]:
!pip install gensim

In [ ]:
import gensim
import gensim.downloader as api

In [ ]:
# Create the tagged document needed for Doc2Vec
def create_tagged_document(list_of_list_of_words):
    for i, list_of_words in enumerate(list_of_list_of_words):
        yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])
train_data = list(create_tagged_document(data))
print(train_data[:1])

[TaggedDocument(words='text', tags=[0])]


In [ ]:
# Init the Doc2Vec model
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
# Build the Volabulary
model.build_vocab(train_data)
# Train the Doc2Vec model
model.train(train_data, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
print(model.infer_vector(['БобышкаБП', '1', ' М', '20Х1,5(17ГС)']))

[-9.0705594e-03  6.4599188e-03  9.3177417e-03  3.8285102e-03
 -5.4775150e-03  6.4540775e-03  8.5979719e-03  5.7223947e-03
  9.7521059e-03 -5.8826036e-03 -5.6333467e-03  3.0546129e-04
 -8.1269592e-03  7.0354925e-03  4.7868686e-03 -1.0465818e-03
 -1.1005597e-03  2.4339340e-03  8.5132131e-03 -8.9769941e-03
 -6.4721461e-03  4.4712499e-03 -9.4511252e-06  6.1965408e-03
  6.1334665e-03 -1.4356803e-03  5.3297998e-03 -8.2028359e-03
  6.2181321e-03 -7.8128250e-03  3.7188784e-03  4.6065841e-03
  7.6870932e-03  2.6736439e-03  2.2319097e-03  4.1340990e-03
  5.3271314e-04 -6.4658346e-03  8.8039003e-03  2.9810872e-03
 -4.2359098e-03 -8.3244760e-03 -5.0227698e-03 -8.5605532e-03
 -1.1408483e-03 -8.7543288e-03 -2.6847403e-03  4.2281612e-05
  2.9002348e-04  7.8156432e-03]


In [ ]:
data['vector'] = data['text'].apply(lambda x: model.infer_vector(x.split(' ')))

In [ ]:
data[['text', 'vector']]

,text,vector
0,Розетка телефонная внешняя двойная RJ-12,"[-0.00259308, 0.009695403, 0.0068313265, -0.00..."
1,Кабель Cisco/CAB-GS-1M= StackWise 1M Stacking ...,"[-0.003520769, -0.009314998, -0.0040105674, -0..."
2,ТЕЛЕФОННЫЙ АППАРАТ SIEMENS 5020,"[0.00016812465, 0.004525493, 0.007519069, 0.00..."
3,Телефон Cisco IP Phone 7911G (CP-7911G),"[-0.0022947127, -0.009718818, -0.009863134, 0...."
4,Cистема производственно технологической громко...,"[0.005713625, 0.00642434, -0.00094878726, 0.00..."
...,...,...
9821,АДАПТЕР REDA 1,"[-0.00017248318, 0.0016618812, -0.006959111, -..."
9822,Антенна,"[0.009293955, -0.006430308, -0.005391588, -0.0..."
9823,ТЕЛЕФОН СОТОВЫЙ SIEMENS M 50,"[0.008981821, 0.0025761696, 0.004844847, 0.003..."
9824,ТЕЛЕФОН СОТОВЫЙ PHILIPS 625,"[6.122438e-06, 0.006656753, 0.0064667976, -0.0..."


ОТБОР ПРИМЕРОВ ДЛЯ РАЗМЕТКИ

In [ ]:
from scipy.spatial import distance

In [ ]:
# чтобы посчитать время поиска одной строки
import time


uniq_vecs = [
    {
    'index': data.index[0],
    'vector': data['vector'].tolist()[0],
    'text': data['text'].tolist()[0]
    } #создаем хранилище для отобранных примеров
] 
vec_distances = {} #записываем, чтобы запомнить расстояние между векторами

print(data['text'].tolist()[0])
for i in range(0, 150): #запускаем цикл на 150 примеров
    tic = time.perf_counter() #запускаем таймер
    
    max_dist = 0 #сюда записываем максимальную дистанцию между векторами
    candidate = {
        'index': data.index[0],
        'vector': data['vector'].tolist()[0],
        'text': data['text'].tolist()[0]
    }
    for ind, item in data.iterrows():
        vec = {
            'index': ind,
            'vector': item['vector'],
            'text': item['text']
        }
        if ind in vec_distances:
            vec_distances[ind] += distance.cosine(vec['vector'], uniq_vecs[-1]['vector'])
        else:
            vec_distances[ind] = distance.cosine(vec['vector'], uniq_vecs[0]['vector'])
        
        if vec_distances[ind] > max_dist:
            max_dist = vec_distances[ind]
            candidate = vec
    uniq_vecs.append(candidate)
    toc = time.perf_counter() #выключаем таймер
    print(candidate['text'], f"Вычисление заняло {toc - tic:0.2f} секунд")


Розетка телефонная внешняя двойная RJ-12
Терминал абонентский R5000-Smntс/5.300.2x63.2x28 InfiNet Вычисление заняло 1.82 секунд
Комплект оборудования и материалов для радиорелейной системы связи пролет Ачимовского м.р.-Чистинного м.р. Спецификация № 0119-15-СН-МНГ от 05.06.2018 г. Вычисление заняло 1.85 секунд
Оборудование радиотехническое МИК-РЛ11 опросный лист № 2 Вычисление заняло 1.82 секунд
Комплект оборудования для переговорной комнаты, в составе: 770001148315 Пульт микшерный PMP500 Behringer-1 шт; 770001148323 Радиосистема двухканальная с двумя ручными передатчиками BLX188E/CVL M17 662-686 MHz Shure-1 шт; 770001004485 Радиосистема двухканальная с двумя ручными передатчиками BLX288E/PG58-M17 Shure-1 шт; 770001148325 Кабель микрофонный распаянный SMXX200/1, XLR(3P)-XLR(3P), длина 1 м Roxton-4 шт; 770001148327 Стойка микрофонная MS022 журавль, цвет черный Roxtone-2 шт; 770001148333 Стойка микрофонная настольная Stand DTS-20 XLine-2 шт; 770001148338 Система акустическая пассивная JR

In [ ]:
uniq_df = pd.DataFrame(uniq_vecs)
ready_data = uniq_df['text']

In [ ]:
ready_data


0               Розетка телефонная внешняя двойная RJ-12
1      Терминал абонентский R5000-Smntс/5.300.2x63.2x...
2      Комплект оборудования и материалов для радиоре...
3      Оборудование радиотехническое МИК-РЛ11 опросны...
4      Комплект оборудования для переговорной комнаты...
                             ...                        
146        Термошкаф ТШ-3В ТУ 26.30.50-077-31006686-2017
147    Система громкоговорящей связи установки ВП №1 ...
148    Радиомодуль абонентский Опросный лист №33-2019...
149    Радиостанция Motorola GM-340 6 каналов 403-470...
150                      Телефон проводной RT-007 Ritmix
Name: text, Length: 151, dtype: object

In [ ]:
ready_data.to_csv(path_or_buf="ready_g28.csv",sep=",", encoding = "utf-8")